## Exploratory Data Analysis

In [8]:
import pandas as pd
import time

import numpy as np
import wrds
import pull_wrds

import config
from pathlib import Path

OUTPUT_DIR = config.OUTPUT_DIR
DATA_DIR = config.DATA_DIR
WRDS_USERNAME = config.WRDS_USERNAME


In [9]:
print(DATA_DIR)

C:/Users/062/GITHUB/DSFinalProject/data


In [12]:
comp = pull_wrds.load_13f(data_dir=DATA_DIR)
crsp = pull_wrds.load_Mutual_Fund(data_dir=DATA_DIR)

In [13]:
display(comp)

,fdate,mgrno,mgrname,typecode,cusip,shares,prc,shrout1
0,1980-03-31,44175.0,HAWAIIAN TRUST LIMITED,1.0,None,NaN,NaN,NaN
1,1980-03-31,44225.0,HEIZER CORPORATION,5.0,None,NaN,NaN,NaN
2,1980-03-31,90415.0,VARIABLE ANNUITY INS CO,2.0,None,NaN,NaN,NaN
3,1980-03-31,14560.0,CENTRAL NATL/RICHMOND,1.0,02473510,41550.0,32.00,28.0
4,1980-03-31,14560.0,CENTRAL NATL/RICHMOND,1.0,00282410,13000.0,36.63,60.0
...,...,...,...,...,...,...,...,...
144884,2017-12-31,82810.0,TEACHERS ADVR INC,2.0,98986X10,23139.0,3.83,24.0
144885,2017-12-31,84900.0,CITIGROUP INC,5.0,98986X10,685.0,3.83,24.0
144886,2017-12-31,90457.0,"VANGUARD GROUP, INC.",5.0,98986X10,599151.0,3.83,24.0
144887,2017-12-31,67780.0,PANAGORA ASSET MANAGEMENT INC.,5.0,98986M10,13800.0,11.07,33.0


In [14]:
display(crsp)

,fdate,mgrco
0,1999-03-31,AAL CAPITAL MANAGEMENT
1,1999-03-31,ASEA BROWN BOVERI INVT MGMT A
2,1999-03-31,ABN AMRO LUXEMBOURG
3,1999-03-31,"ACADIAN ASSET MANAGEMENT, INC"
4,1999-03-31,"ACADEMY CAPITAL MGMT, INC."
...,...,...
153576,2017-12-31,ADULA CAPITAL AG
153577,2017-12-31,WPS ADVR LIMITED (BERMUDA)
153578,2017-12-31,"URSUS-3 CAPITAL, AV, S.A"
153579,2017-12-31,VERDE ASSET MANAGEMENT S.A.


In [15]:
comp.rename(columns={'fdate': 'date'}, inplace=True)
comp['date'] = pd.to_datetime(comp['date'])
grouped = comp.groupby(['mgrno', 'date'])

In [19]:
comp_sorted = comp.sort_values(by=['mgrno', 'date'])

In [21]:
display(comp_sorted)

,date,mgrno,mgrname,typecode,cusip,shares,prc,shrout1
353453,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,03783310,23100.0,40.94,135.0
355777,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,00282410,3500.0,49.00,1518.0
360280,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,03948310,8792.0,17.13,593.0
361748,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,00792410,2641.0,122.25,580.0
361925,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,01747510,1078.0,46.63,113.0
...,...,...,...,...,...,...,...,...
47090,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,88284810,77300.0,45.75,220.0
47106,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,90478450,33600.0,105.75,160.0
47123,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,91191010,28500.0,47.50,108.0
47179,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,93114210,147800.0,26.25,2300.0


In [22]:
nunique_dates = grouped['date'].nunique().sum()
nunique_mgrno = comp['mgrno'].nunique()

print(f"Number of unique dates: {nunique_dates}")
print(f"Number of unique 'mgrno': {nunique_mgrno}")


Number of unique dates: 306243
Number of unique 'mgrno': 8954


In [23]:
nunique_dates = comp_sorted['date'].nunique()
nunique_mgrno = comp_sorted['mgrno'].nunique()

print(f"Number of unique dates: {nunique_dates}")
print(f"Number of unique 'mgrno': {nunique_mgrno}")

Number of unique dates: 152
Number of unique 'mgrno': 8954


In [24]:
(2018-1980)*4

152

Quarterly reports

In [25]:
nunique_mgrco = crsp['mgrco'].nunique()
print(f"Number of unique 'mgrco': {nunique_mgrco}")

Number of unique 'mgrco': 11875


Need to match companies?

In [26]:
unique_typecodes = comp_sorted['typecode'].unique()
print(unique_typecodes)

[ 4.  5.  3.  2.  1. nan]


Split bins of years